# Práctico 2: Funciones de activación, _Loss functions_, Hiperparámetros, _Vanishing/Exploding Gradients_ e Inicialización

En este cuaderno trabajaremos con MNIST y ejemplos de juguete para comprender los efectos de usar diferentes _losses_, cambiar las funciones de activación, variar ancho, profundidad y _learning rate_, _vanishing_ y _exploding gradients_ y diferentes estrategias de inicialización.

Arrancamos cargando algunas bibliotecas básicas

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
#esto que viene es para tener una ventana interactiva
import ipywidgets as widgets
from IPython.display import display
#esto de abajo son optimizadores de pytorch
import torch.optim as optim

## El dataset MNIST

In [ ]:
#Ahora importamos los datasets que tiene pytorch y funciones utilitarias
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
# Preprocesamiento MNIST
transform = transforms.ToTensor()
train_full = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
print (f"Tamaño del training set: {len(train_full)}, tamaño del test set: {len(test_ds)}, forma de los datos: {train_full[0][0].shape}")

## Crear validation set para seguir el entrenamiento
El training set lo partimos sacándole una parte para hacer validación dentro del entrenamiento. Eso no se va a usar para calcular los gradientes.

In [ ]:
# Separar un pequeño conjunto para la validación
train_size = int(len(train_full) * 0.9)
val_size = len(train_full) - train_size
train_ds, val_ds = random_split(train_full, [train_size, val_size])

# Instanciar clases utilitarias de PyTorch que nos permiten leer la carpeta data
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)# ----- Definir red profunda -----
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)#

# Imprimimos la forma del primer lote
batch = next(iter(train_loader))
batch[0].shape, batch[1].shape

## Visualicemos el dataset

In [ ]:
# --- Función para mostrar imagen por índice ---
def show_example(index):
    image, label = train_full[index]
    plt.imshow(image.squeeze(), cmap="gray")
    plt.title(f"Label: {label}")
    plt.axis("off")
    plt.show()

# --- Slider interactivo ---
slider = widgets.IntSlider(value=0, min=0, max=len(train_full)-1, step=1, description="Index")
widgets.interact(show_example, index=slider)

## Para hacer:

In [ ]:
#copiando el código y modificando apropiadamente pueden ver las imágenes del test set.

## Primer intento
Primero vamos a hacer una red con una capa oculta, con funciones de activación sigmoides y una salida vectorial que buscaremos aproximar en mínimos cuadrados a un vector que tiene un "_one hot encoding_" de las etiquetas con números. Esta NO es la mejor manera de clasificar dígitos (o cualquier otra cosa), pero arranquemos ahí.

In [ ]:
# Red simple con ancho parametrizable
class SimpleNet(nn.Module):
    def __init__(self, width=128):
        super().__init__()
        #self.capa0=nn.Flatten()   # aplanar la imagen 28x28 -> vector
        in_dim = 28 * 28
        #la entrada va a una capa oculta con ancho width
        self.capa1=nn.Linear(in_dim, width)
        #la capa oculta va a una capa de salida que recibe width entradas tiene 10 unidades de ancho
        self.capa2=nn.Linear(width, 10)

    def forward(self, x):
        x = x.view(x.size(0),-1)
        #x=self.capa0(x)
        #acá ponemos las sigmoides
        x=torch.sigmoid(self.capa1(x))
        x=torch.sigmoid(self.capa2(x))
        return x


In [ ]:
def init_normal(m):
     if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0.0, std=1.0) # Inicialización Normal
        nn.init.zeros_(m.bias)# Bias en cero


# Crear el modelo
mlp = SimpleNet(width=256)

# Aplicar inicialización personalizada
mlp.apply(init_normal)

# Imprimir arquitectura
print(mlp)


In [ ]:
# Vamos a empezar con MSELoss y para eso la salida tengo que cambiarla a vectrores one hot, con 1 en el indice clase-1
# O sea, si el dígito es 3, la salida deseada es un vector salida=[0 0 0 1 0 0 0 0 0 0]


In [ ]:
import torch.nn.functional as F

def train_model0(model, train_loader, val_loader, optimizer, epochs=5, device="cpu"):
    model.to(device)
    history = {"train_loss": [], "val_loss": [], "val_acc": []}

    for epoch in range(epochs):
        # -------- TRAIN --------
        model.train()
        total_loss = 0
        for X, y in train_loader:
            # convertir labels a one-hot
            y_onehot = F.one_hot(y, num_classes=10).float()
            X, y_onehot = X.to(device), y_onehot.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = F.mse_loss(out, y_onehot)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        history["train_loss"].append(avg_loss)

        # -------- EVAL --------
        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                # one-hot para el loss
                y_onehot = F.one_hot(y, num_classes=10).float()
                X, y_onehot = X.to(device), y_onehot.to(device)

                out = model(X)  # 👈 recalculamos forward
                loss = F.mse_loss(out, y_onehot)
                val_loss += loss.item()

                # accuracy con labels enteros
                pred = out.argmax(dim=1).cpu()
                correct += (pred == y).sum().item()
                total += y.size(0)

        history["val_loss"].append(val_loss / len(val_loader))
        history["val_acc"].append(correct / total)

        print(f"Epoch {epoch+1}: "
              f"train_loss={avg_loss:.4f}, "
              f"val_loss={history['val_loss'][-1]:.4f}, "
              f"val_acc={history['val_acc'][-1]:.4f}")
    return history


In [ ]:
import numpy as np
# Especificamos el optimizador
optimizer_mlp = torch.optim.Adam(mlp.parameters(), lr=1e-3)

# Reproducibilidad
torch.manual_seed(123)
np.random.seed(123)

# Iteramos sobre una cierta cantidad de épocas
history = train_model0(mlp, train_loader, val_loader, optimizer_mlp, 15)


In [ ]:
plt.plot(history["train_loss"])
plt.plot(history['val_loss'])

In [ ]:
device="cpu"

# --- función para mostrar un ejemplo y pasarlo por la red ---
def show_prediction(index):
    modelo=mlp
    modelo.eval()
    image, label = test_ds[index]  # un ejemplo del test set
    x = image.unsqueeze(0).to(device)   # añadir batch dimension [1, 1, 28, 28]

    with torch.no_grad():
        output = modelo(x)
        probs = torch.softmax(output, dim=1).cpu().numpy().flatten() #esto lo explicamos después
        pred = probs.argmax()

    # --- mostrar imagen ---
    plt.imshow(image.squeeze(), cmap="gray")
    plt.axis("off")
    plt.title(f"Etiqueta real: {label}\nPredicción: {pred}")
    plt.show()

    # --- mostrar probabilidades como vector ---
    print("Vector de salida (softmax):")
    for i, p in enumerate(probs):
        print(f"{i}: {p:.3f}")

# --- slider interactivo ---
slider = widgets.IntSlider(value=0, min=0, max=len(test_ds)-1, step=1, description="Index")
widgets.interact(show_prediction, index=slider)

Evaluemos ahora la precisión en el test set.

In [ ]:
def evaluate_accuracy(model, test_loader, device="cpu"):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():

        for X, y in test_loader:
            X, y = X.to(device), y.to(device)

            # forward
            out = model(X)

            # predicciones (clase con mayor logit)
            pred = out.argmax(dim=1)
            #prediccion clase con softmax
            #print(out)
            #probs = torch.softmax(out, dim=1).cpu().numpy().flatten() #esto lo explicamos después
            #pred = probs.argmax()

            correct += (pred == y).sum().item()
            total += y.size(0)



    acc = correct / total
    print(f"Accuracy en test set: {acc:.4f}")
    return acc


In [ ]:
acc = evaluate_accuracy(mlp, test_loader, device=device)

## Un _Loss_ mejor: CrossEntropyLoss

Vamos a introducir paulatinamente diferentes modificaciones para mejorar nuestro modelo.
Lo primero es que **no es una buena idea buscar vectores cercanos al _one-hot encoding_ de las categorías** y usar directamente las categorías.
Para eso hay que cambiar la función de pérdida. Si tenemos varias categorías, nos gustaría maximizar la probabilidad de asignar la categoría correcta a los ejemplos. Si asumimos que hay una distribución de probabilidad real queremos minimizar la "distancia" a esa distribución de probabilidad. Las comillas van en el sentido de que esa distancia no es una distancia en toda la regla, sino que se usa la divergencia de Kullback-Leibler (que no cumple uno de los axiomas para la distancia). La pérdida al clasificar un ejemplo puede verse como la diferencia de información necesaria para obtener la distribución correcta en los ejemplos.

## Función de activación _softmax_
Para eso primero vamos a modificar un poco la red. En particular deberíamos agregar una función softmax a la salida, en lugar de una sigmoide. La función _softmax_ nos dá como resultado una salida que tiene probabilidades en el sentido de que las componentes suman 1 y se pueden interpretar como la probabilidad de la componente i-ésima. Si tengo 10 componentes, hacemos que la salida nos dé la probabilidad de cada una de las clases en cada ejemplo.
Como vamos a usar CrossEntropy Loss, este ya incluye log softmax en el cálculo de la pérdida. Alcanza por lo tanto con dejar un vector de diez componentes en la salida lineal.

In [ ]:
# Red simple con ancho parametrizable
class SNet(nn.Module):
    def __init__(self, width=128):
        super().__init__()
        #self.capa0=nn.Flatten()   # aplanar la imagen 28x28 -> vector
        in_dim = 28 * 28
        #la entrada va a una capa oculta con ancho width
        self.capa1=nn.Linear(in_dim, width)
        #la capa oculta va a una capa de salida que recibe width entradas tiene 10 unidades de ancho
        self.capa2=nn.Linear(width, 10)


    def forward(self, x):
        x = x.view(x.size(0),-1)
        #x=self.capa0(x)
        #acá ponemos las sigmoides
        x=torch.sigmoid(self.capa1(x))
        #sacamos la última sigmoide y lo dejamos como una salida lineal; en el entrenamiento el CrossEntropyLoss aplica un LogSoftmax
        x=self.capa2(x)
        return x


In [ ]:
# Especificamos el criterio para medir la pérdida
criterion = nn.CrossEntropyLoss()
import torch.nn.functional as F
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs=5, device="cpu"):
    model.to(device)
    history = {"train_loss": [], "val_loss": [], "val_acc": []}
    for epoch in range(epochs):
        # -------- TRAIN --------
        model.train()
        total_loss = 0
        for X, y in train_loader:
            # sacamos lo de convertir labels a one-hot
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        history["train_loss"].append(avg_loss)

        # -------- EVAL --------
        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                # sin one-hot
                X, y = X.to(device), y.to(device)
                out = model(X)  # 👈 recalculamos forward
                loss = criterion(out, y)
                val_loss += loss.item()
                # accuracy con labels enteros
                pred = out.argmax(dim=1).cpu()
                correct += (pred == y).sum().item()
                total += y.size(0)
        history["val_loss"].append(val_loss / len(val_loader))
        history["val_acc"].append(correct / total)

        print(f"Epoch {epoch+1}: "
              f"train_loss={avg_loss:.4f}, "
              f"val_loss={history['val_loss'][-1]:.4f}, "
              f"val_acc={history['val_acc'][-1]:.4f}")
    return history

In [ ]:
# Crear el modelo
mlp2 = SNet(width=256)

# Aplicar inicialización personalizada
mlp2.apply(init_normal)

# Imprimir arquitectura
print(mlp2)


In [ ]:
# Especificamos el optimizador
optimizer_mlp = torch.optim.Adam(mlp2.parameters(), lr=1e-3)

# Reproducibilidad
torch.manual_seed(123)
np.random.seed(123)

# Iteramos sobre una cierta cantidad de épocas
history_CE= train_model(mlp2, train_loader, val_loader, optimizer_mlp, criterion,  15)

In [ ]:
plt.plot(history_CE["train_loss"])
plt.plot(history_CE['val_loss'])

In [ ]:
acc2 = evaluate_accuracy(mlp2, test_loader, device=device)

# Profundidad o ancho?
Esta red tiene sólo una capa oculta de 256 unidades. Capaz que se pueden agregar más unidades. Se supone que cualquier función puede representarse con un perceptron con una sóla capa oculta (de dimensión correcta). Ahí tenemos una decisión que tomar; ponemos más capas o le agregamos más unidades a la capa. Exploremos primero esto.

Comparamos:
- Una red llana y ancha (1 _hidden layer_ y varias unidades)
- Una red profunda y estrecha (muchas capas con pocas unidades)

Para hacer la comparación usamos el problema de paridad: Decir 1 si el input tiene un número par de 1s y 0 si es impar.

Vamos a greficar las curvas de aprendizaje. La idea es que la red profunda puede demorar más o no, pero aprende mejor.

Veremos:
- **Curvas de aprendizaje** en paridad de 8-bits
- **Comportamiento de scaling** para diferentes tamaños de entrada (4, 8, 12, 16 bits)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
def generate_parity_torch(n_bits=8, n_samples=1000):
    X = torch.randint(0, 2, (n_samples, n_bits)).float()
    y = (X.sum(dim=1) % 2).long()
    return X, y

Vamos a crear dos redes, una con 128 unidades en la capa oculta (son menos porque el problema es menor que el de los dígitos) y otra que tiene la misma cantidad de unidades en cuatro capas. Usamos otra función de activación, pero eso lo hablamos despúes.

In [ ]:
class ShallowNet(nn.Module):
    def __init__(self, input_dim, hidden_units=128):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_units)
        self.out = nn.Linear(hidden_units, 2)
    def forward(self, x):
        return self.out(torch.relu(self.fc1(x)))#

class DeepNet(nn.Module):
    def __init__(self, input_dim, hidden_units=32, depth=4):
        super().__init__()
        layers = [nn.Linear(input_dim, hidden_units), nn.ReLU()]
        for _ in range(depth-1):
            layers.append(nn.Linear(hidden_units, hidden_units))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_units, 2))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

In [ ]:
def train_net(model, train_loader, test_loader, epochs=40, lr=1e-3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_losses, test_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for Xb, yb in train_loader:
            optimizer.zero_grad()
            outputs = model(Xb)
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_losses.append(total_loss / len(train_loader))

        # Evaluate
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for Xb, yb in test_loader:
                preds = model(Xb).argmax(dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)
        test_accuracies.append(correct/total)
    return train_losses, test_accuracies


In [ ]:

# Data
X_train, y_train = generate_parity_torch(8, 2000)
X_test, y_test = generate_parity_torch(8, 500)
tr_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
tst_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)

# Models
shallow_net = ShallowNet(input_dim=8, hidden_units=128)
deep_net = DeepNet(input_dim=8, hidden_units=32, depth=4)

# Train
shallow_losses, shallow_acc = train_net(shallow_net, tr_loader, tst_loader)
deep_losses, deep_acc = train_net(deep_net, tr_loader, tst_loader)

# Plot
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(shallow_losses, label="Llana")
plt.plot(deep_losses, label="Profunda")
plt.xlabel("Epochs"); plt.ylabel("Training Loss"); plt.title("Training Curves"); plt.legend()

plt.subplot(1,2,2)
plt.plot(shallow_acc, label="Shallow")
plt.plot(deep_acc, label="Deep")
plt.xlabel("Epochs"); plt.ylabel("Test Accuracy"); plt.title("Test Accuracy Curves"); plt.legend()
plt.show()

print(f"Final Shallow Accuracy: {shallow_acc[-1]:.3f}")
print(f"Final Deep Accuracy: {deep_acc[-1]:.3f}")

La relevancia de la profundidad se ve mejor si tenemos problemas más complejos. Acá variamos el tamaño del problema y vemos si hay más ventajas de la profunidad cuanto más grande es el problema.

In [ ]:
bit_sizes = [4, 8, 12, 16]
shallow_final_acc, deep_final_acc = [], []

for n_bits in bit_sizes:
    X_train, y_train = generate_parity_torch(n_bits, 6000)
    X_test, y_test = generate_parity_torch(n_bits, 500)
    tr_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    tst_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)

    shallow_net = ShallowNet(input_dim=n_bits, hidden_units=128)
    deep_net = DeepNet(input_dim=n_bits, hidden_units=32, depth=4)

    _, shallow_acc = train_net(shallow_net, tr_loader, tst_loader, epochs=40)
    _, deep_acc = train_net(deep_net, tr_loader, tst_loader, epochs=40)

    shallow_final_acc.append(shallow_acc[-1])
    deep_final_acc.append(deep_acc[-1])
    print(f"{n_bits}-bit → Shallow: {shallow_acc[-1]:.3f}, Deep: {deep_acc[-1]:.3f}")

# Plot scaling
plt.figure(figsize=(6,5))
plt.plot(bit_sizes, shallow_final_acc, marker='o', label="Shallow")
plt.plot(bit_sizes, deep_final_acc, marker='o', label="Deep")
plt.xlabel("Number of bits (input size)")
plt.ylabel("Final Test Accuracy")
plt.title("Scaling: Shallow vs Deep on Parity")
plt.legend()
plt.show()


## Volvemos al _dataset_ MNIST
Ahora vamos a crear una red profunda, a ver si mejoramos el accuracy

In [ ]:
# ----- Definir red profunda -----
class DeepMLP(nn.Module):
    def __init__(self, in_dim, out_dim, width=64, depth=4):
        super().__init__()
        layers = [nn.Linear(in_dim, width)]
        for i in range(1,depth):
            layers.append(nn.Linear(width, width))
        layers.append(nn.Linear(width, out_dim))  # capa de salida
        self.layers = nn.ModuleList(layers)
    def forward(self, x):
        x = x.view(x.size(0),-1)
        for layer in self.layers[:-1]:
            x = F.sigmoid(layer(x))
        out = self.layers[-1](x)  # salida (sin activación para usar softmax)
        return out# ----- Definir red profunda -----


In [ ]:
# Crear el modelo
mlp3 = DeepMLP(28*28, 10, 64,4)

# Imprimir arquitectura
print(mlp3)

In [ ]:
import numpy as np
# Especificamos el optimizador
optimizer_mlp = torch.optim.Adam(mlp3.parameters(), lr=1e-3)

# Reproducibilidad
torch.manual_seed(13)
np.random.seed(13)

# Iteramos sobre una cierta cantidad de épocas
history_deep= train_model(mlp3, train_loader, val_loader, optimizer_mlp, criterion,  15)

In [ ]:
plt.plot(history_deep["train_loss"]) # linea azul
plt.plot(history_deep['val_loss']) # linea naranja

In [ ]:
acc3 = evaluate_accuracy(mlp3, test_loader, device=device)

Si comparan con el valor de accuracy anterior, no parece que acá esté funcionando lo de poner más capas. Seguramente hay que poner más neuronas, pero la caída lenta del error y la no generalización (noten que el error de validación no baja mucho más allá de la época 10) hace pensar que hay algo más.
Sugaremente esto es una instancia del _**vanishing gradient problem**_.

## Cambiando la función de activación
Hagamos una red en donde usamos funciones ReLU

In [ ]:
# ----- Definir red profunda -----
class DeepMLPR(nn.Module):
    def __init__(self, in_dim, out_dim, width=64, depth=4):
        super().__init__()
        layers = [nn.Linear(in_dim, width)]
        for i in range(1,depth):
            layers.append(nn.Linear(width, width))
        layers.append(nn.Linear(width, out_dim))  # capa de salida
        self.layers = nn.ModuleList(layers)
    def forward(self, x):
        x = x.view(x.size(0),-1)
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = self.layers[-1](x)  # salida (sin activación para usar softmax)
        return out# ----- Definir red profunda -----


In [ ]:
# Crear el modelo
mlp4 = DeepMLPR(28*28, 10, 64,4)

# Imprimir arquitectura
print(mlp4)

In [ ]:
# Especificamos el optimizador
optimizer_mlp = torch.optim.Adam(mlp4.parameters(), lr=1e-3)

# Reproducibilidad
torch.manual_seed(13)
np.random.seed(13)

# Iteramos sobre una cierta cantidad de épocas
history_deepR= train_model(mlp4, train_loader, val_loader, optimizer_mlp, criterion,  15)

In [ ]:
plt.plot(history_deepR["train_loss"])
plt.plot(history_deepR['val_loss'])

In [ ]:
acc4 = evaluate_accuracy(mlp4, test_loader, device=device)

Esto parece mejorar. También deberíamos mirar la inicialización

## Cambiando la inicialización

En la década 2010-2020 aparecieron varios trabajos que exploraban varias formas de iniciar los pesos de la red, lo que tenía un efecto sobre el entrenamianeto. Probemos qué pasa con un ejemplo de juguete.

In [ ]:
# ----- Definir red profunda -----
class DeepMLP_i(nn.Module):
    def __init__(self, init_type="xavier", depth=15, width=128, in_dim=100, out_dim=10):
        super().__init__()
        layers = []
        for i in range(depth):
            layers.append(nn.Linear(in_dim if i == 0 else width, width))
        layers.append(nn.Linear(width, out_dim))  # capa de salida
        self.layers = nn.ModuleList(layers)
        self.init_type = init_type
        self.reset_parameters()

    def reset_parameters(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                if self.init_type == "xavier":
                    nn.init.xavier_uniform_(layer.weight)
                elif self.init_type == "he":
                    nn.init.kaiming_uniform_(layer.weight, nonlinearity="relu")
                elif self.init_type == "orthogonal":
                    nn.init.orthogonal_(layer.weight, gain=torch.sqrt(torch.tensor(2.0)))
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        activations = []
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
            activations.append(x.detach().std().item())
        out = self.layers[-1](x)  # salida (sin ReLU)
        return out, activations

# ----- Función para obtener estadísticas -----
def probe_stats(net, batch_size=64, in_dim=100):
    #estos son los datos
    x = torch.randn(batch_size, in_dim)
    out, activations = net(x)
    # Backprop para ver gradientes
    loss = out.pow(2).mean()
    loss.backward()
    grad_norms = [layer.weight.grad.std().item() for layer in net.layers if hasattr(layer.weight, "grad")]
    return activations, grad_norms


In [ ]:
# ----- Experimento -----
depth = 15
inits = ["xavier", "he", "orthogonal"]
results = {}

for init in inits:
    net = DeepMLP_i(init_type=init, depth=depth, width=128)
    activations, grads = probe_stats(net)
    results[init] = {"activations": activations, "grads": grads}


In [ ]:
# ----- Graficar -----
plt.figure(figsize=(12,5))

# Activaciones
plt.subplot(1,2,1)
for init in inits:
    plt.plot(results[init]["activations"], label=init)
plt.xlabel("Capa oculta (0 = primera)")
plt.ylabel("Std activaciones")
plt.title("Propagación de activaciones")
plt.legend()
plt.grid(True)

# Gradientes
plt.subplot(1,2,2)
for init in inits:
    plt.plot(results[init]["grads"], label=init)
plt.xlabel("Capa (0 = primera oculta)")
plt.ylabel("Std gradientes")
plt.title("Propagación de gradientes")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

En el caso de nuestra última red, lo que parece funcionar mejor (con unidades ReLU, es la inicialización de He. POngamos eso y vemos qué pasa.

## De vuelta a MNIST

In [ ]:
# ----- Definir red profunda -----
class DeepMLPI(nn.Module):
    def __init__(self, in_dim, out_dim, width=64, depth=4):
        super().__init__()
        layers = [nn.Linear(in_dim, width)]
        for i in range(1,depth):
            layers.append(nn.Linear(width, width))
        layers.append(nn.Linear(width, out_dim))  # capa de salida
        self.layers = nn.ModuleList(layers)
        self.reset_parameters()

    def reset_parameters(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight, nonlinearity="relu")
                nn.init.zeros_(layer.bias)
    def forward(self, x):
        x = x.view(x.size(0),-1)
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = self.layers[-1](x)  # salida (sin activación para usar softmax)
        return out# ----- Definir red profunda -----

In [ ]:
# Crear el modelo
mlp5 = DeepMLPI(28*28, 10, 64,4)


In [ ]:
# Especificamos el optimizador
optimizer_mlp = torch.optim.Adam(mlp5.parameters(), lr=1e-3)

# Reproducibilidad
torch.manual_seed(234)
np.random.seed(234)

# Iteramos sobre una cierta cantidad de épocas
history_deepI= train_model(mlp5, train_loader, val_loader, optimizer_mlp, criterion,  15)

In [ ]:
plt.plot(history_deepI["train_loss"])
plt.plot(history_deepI['val_loss'])

In [ ]:
acc5 = evaluate_accuracy(mlp5, test_loader, device=device)